## Names of group members (max 3):
    
    Dmitri Shostakovich
    Gustav Mahler
    Johannes Brahms



## $$\mbox{Logistic Regression.}$$


In [ ]:
# %load ../../standard_import.txt
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from scipy.optimize import minimize

from sklearn.preprocessing import PolynomialFeatures

from IPython.display import display, Math, Latex

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_seq_items', None)
 
#%config InlineBackend.figure_formats = {'pdf',}
%matplotlib inline

import seaborn as sns
sns.set_context('notebook')
sns.set_style('white')

In [ ]:
def loaddata(file, delimeter):
    data = np.loadtxt(file, delimiter=delimeter)
    print('Dimensions: ',data.shape)
    print(data[1:6,:])
    return(data)

In [ ]:
def plotData(data, label_x, label_y, label_pos, label_neg, axes=None):
    ## the data gives the grades for two exams; 
    ## the last entry in the data is a 0 or 1: 
    ##   rejected or accepted for a certain programme
    ## make a scatter plot of the data such that 
    ## a 1 gets a black plus, the 0 a yellow (not too) large dot
    if axes == None:
        axes = plt.gca()

        ## ...

### $$\mbox{Logistic regression.}$$

#### $$ \mbox{Logistic regression hypothesis:} \hspace{1cm} h_{\theta}(x) = \sigma(\theta^{T}x), ~~~ \sigma(z)=\frac{1}{1+e^{−z}} $$

In [ ]:
def sigmoid(z):
    ### return

#### $$\mbox{Prediction.}$$

In [ ]:
def predict(theta, X, threshold = 0.5):
    ### return

###  $$\mbox{Regularized logistic regression.}$$

In [ ]:
data2 = loaddata('data/ex2data2.txt', ',')

In [ ]:
## y = 
## X =

In [ ]:
plotData(data2, 'Microchip Test 1', 'Microchip Test 2', 'y = 1', 'y = 0')

####   $$\mbox{Polynomials.}$$

In [ ]:
# Note that this function inserts a column with 'ones' in the design matrix for the intercept.
poly = PolynomialFeatures(6)
XX = poly.fit_transform(data2[:,0:2])
XX.shape

#### $$\mbox{Regularized Cost Function:}\hspace{1cm} J(\theta) = -\frac{1}{m}\big((\,\ln\,(\sigma(X\theta))^Ty+(\,\ln\,(1-\sigma(X\theta))^T(1-y)\big) + \frac{\lambda_{reg}}{2m}\sum_{j=1}^{n}\theta_{j}^{2}$$

In [ ]:
def costFunctionReg(theta, X, y, reg):
    ### J = ...
    if np.isnan(J):
        return(np.inf)
    return(J)


#### $$\mbox{Partial derivative:}\hspace{1cm}  \frac{\partial J(\theta)}{\partial\theta_{j}} = \frac{1}{m} X^T(\sigma(X\theta)-y) + \frac{\lambda_{reg}}{m}\theta_{j}$$
##### $$\text{Note: intercept parameter } \theta_{0} \text{ is not to be regularized}$$

In [ ]:
def gradientReg(theta, X, y, reg):
    ## return ..

In [ ]:
initial_theta = np.c_[np.zeros(XX.shape[1])]
costFunctionReg(initial_theta, XX, y, 1)

In [ ]:
def gradientDescentReg(theta, X, y, reg, alpha = 0.001, num_iters = 50000):
    M = y.size
    J = 0
    for iter in np.arange(num_iters):
        ## theta = ...
        ## J = ...
    return(theta, J)

In [ ]:
## we vary the value of the regulaization parameter, reg
i = 0
fig, axes = plt.subplots(1,3, sharey = True, figsize=(17,5))  
for reg in {100, 1, 0.01}:
    theta = initial_theta
    print('initial cost: ',costFunctionReg(theta, XX, y, reg))
    theta , J = gradientDescentReg(theta, XX, y, reg)
    print('final cost: ',costFunctionReg(theta, XX, y, reg))
    print('theta = ', theta.T)
    accuracy = 100*(sum(predict(theta, XX) == y))/y.size
    plotData(data2, 'Microchip Test 1', 'Microchip Test 2', 'y = 1', 'y = 0', axes.flatten()[i])
    # Plot decisionboundary
    x1_min, x1_max = X[:,0].min(), X[:,0].max(),
    x2_min, x2_max = X[:,1].min(), X[:,1].max(),
    xx1, xx2 = np.meshgrid(np.linspace(x1_min, x1_max), np.linspace(x2_min, x2_max))
    h = sigmoid(np.dot(poly.fit_transform(np.c_[xx1.ravel(), xx2.ravel()]), theta))
    h = h.reshape(xx1.shape)
    axes.flatten()[i].contour(xx1, xx2, h, [0.5], linewidths=1, colors='g'); 
    axes.flatten()[i].set_title('Train accuracy {}% with reg = {}'.format(np.round(accuracy[0], decimals=2), reg))
    i = i + 1
 

In [ ]:
## what is your comment on these results